# Continuous Assessment 1


## Additional Libraries and Tools


### [Github Repo](https://github.com/Ri-Dearg/data-analytics-dbs/)
This file and its commit history [can be found by clicking here](https://github.com/Ri-Dearg/data-analytics-dbs/blob/main/programming/ca1/ca1.ipynb).

### [Colorama](https://pypi.org/project/colorama/)
Used to give feedback in varied colours in the terminal.

### [Pylint](https://www.pylint.org)
Used as a linter for code quality control.

### [Ruff](https://docs.astral.sh/ruff/)
Used as both a linter and a formatter. Configuration has been adjusted to conform to PEP8 standard.  



In [2]:

# Necessary imports.
from pathlib import Path

# Add colour to text.
from colorama import Fore, Style

## Utilities
Tools for input validation used throughout the notebook.


### Code Explanation
These functions were created to validate numeric input, display an error message and request the user try again with the correct format. Many questions require numeric input, and also that the number is positive. While doing the questions, I had repeated the same code numerous times, and so decided to create a utility function to be used throughout the notebook.  
  
- `validate_input_num` is the main function, taking in the question, and whether the number must be an int or a float. It uses a `while` loop and `try / except` to deal with invalid input. Requesting correct input until the user inputs it or exits. 
   
- `parse_input` tries to convert the string input to an int or float. Failing with raise a `ValueError`.  
- `positive_number_validation` checks that the number is greater than zero. If it isn't, it raises a `ValueError`, triggering the `except` block of the function.

In [6]:
def positive_number_validation(num: float) -> None:
    """Ensure number is positive, throw error if not."""
    if num < 0:
        raise ValueError


def parse_input(value: str, *, is_int: bool) -> int | float:
    """Parse string as an int or float.

    Args:
        value (str): user input
        is_int (bool): sets if it is an int or float.

    Returns:
        int|float: Converted user input.

    """
    return int(value) if is_int else float(value)


def validate_input_num(user_input: str, *, is_int: bool) -> str:
    """Validate inputs to check that they are positive numbers.

    Args:
        user_input (str): user_input for the input.
        is_int (bool): defines whether we are dealing with an integer or float.

    Returns:
        str: return the user_input if everything passes.

    """
    # Define error messages for ints or floats
    if is_int:
        err_msg = 'Please input positive whole numbers, ex. "12".'
    else:
        err_msg = (
            'Please input positive numbers with two decimals, ex. "12.95".'
        )

    # Use loop to ask for input again if the input is invalid
    while True:
        try:
            value = input(f'{user_input} ')
            test_num = parse_input(value, is_int=is_int)
            positive_number_validation(test_num)
        except ValueError:
            # Give feedback with the err_msg
            print(Fore.RED + err_msg + Style.RESET_ALL + '\n')
        else:
            # Return value if input is valid
            return test_num


## CA1 Main Assessment

### Question

Details of assignment.To create files to write data to:

- consider a file product.txt containing list of product names
- consider a file quantity.txt containing the quantity of products
- consider a file purchase_prices.txt containing the purchasing price of the products purchased
- consider a file sales_prices.txt containing the price of the products sold

Create these four files, quantity.txt, sale_price.txt, purchase_price.txt and product.txt  
Request input for how many items (N), which is the number of expected rows of data to enter?  
Request input for product name, the quantity of the product, the purchased price and sales price.  
Supply the value at each input for product name, quantity and price for sale and purchase and write the inputted values to the respective files line by line using values from the below table.

Data to write to respective files for calculation of profit.

| **Product** | **Quantity** | **Purchase_price** | **Sale_price** |
| ----------- | ------------ | ------------------ | -------------- |
| Mattress    | 420          | $18                | $29            |
| Bed         | Six          | $433               | $554           |
| Pillow      | 2214         | $1263              | $1329          |
| Door        | 55           | $2344              | $2614          |
| Table       | 3764         | $122               | $2422          |
| Chairs      | 37           | $1233              | $1629          |

**Do not correct any error on the data provided but program defensively so that it does not break your program when it encounters this error.**

**After creating the files:**

Read the values from the files to calculate profit:  
 Revenue = Quantity\*Sale_price  
 Cost = quantity\*purchase_price

Note if revenue > $55000 then a cashback of $3000 is given so revenue is less $3000  
If revenue > $22000 and <= $55000 then cashback of $150 is given so revenue is less by $150  
But If revenue <=22000 then cashback of $0 is given

The calculated total cost and total revenue (exclude cashback from revenue where applicable)

Profit = total revenue – total cost.

Report the product name, total revenue , total cost and profit.  
Check if final profit > 0 then print a message.  
If final profit = 0 then print a message  
If final profit < 0 then print a message

### Code Explanation

I built a sales calculator that uses different functions for different goals. The idea was to follow the practice of "Separation of Concerns". Initially, it was a group of different functions that worked together. However, once I had completed the functions, I structured it into a class so that all the related functions were contained in a single space, and variables that were used more universally, like `FILE_NAMES`, could be more easily accessed. As requested by the question, it uses files and loops to achieve its goals. More specific explanations for what the code is doing can be found in the comments.
  
The function uses one external utility function, `validate_input_num` because it is used by other functions in the notebook as well.  

- The `SalesCalculator` class initialises with some constants to be used throughout other methods. The main code is in the `run` method.  
  
- The `run` method asks for how many items are being input, and then calls the other methods in order before displaying a final message at the end.

- `check_new_item` is used by `write_files` to confirm the product han't already been entered.

- `write_files` runs for each item to be entered into the text files. It asks each question in order and writes the data to the files. It will always append new items to the file instead of overwriting the files on a new run.

- `create_dict` runs through each line of the text files and structures a dictionary from their values. This uses the specific structure of `FILE_NAMES` to create the dictionary. Finally, it returns the dictionary for calculations.  
Changing the order of the constants or an invalid structure in the files could break the code. The question specifically requests I access the text files after they are written to continue the calculation, but I would have done things differently. I would have created teh dictionary at the same moment the user inputs the information to be written to the file. This would have avoided accessing the files twice unnecessarily.  

- `calculate_product_stats` is run by `compile_stats` to calculate revenue, cost, and profit of each item in the dictionary and returns them. The specific structure of the dictionary is very important here.

- `compile_stats` adds up all the values from the dictionary and displays them to the user. Uses colorama for better feedback.

In [4]:
class SalesCalculator:
    """Tool for inputting product details and calculating profit or loss."""

    def __init__(self) -> None:
        """Constants for declaring cashback and accessing file names."""
        # Constants for cashback to avoid using magic values
        self.UPPER_CASHBACK = 55000
        self.LOWER_CASHBACK = 22000

        # Dictionary of file names and input questions for each
        self.FILE_NAMES = {
            'product': 'What is the name of the product?',
            'quantity': 'How many of this product is there?',
            'purchase_price': 'How much does it cost to purchase?',
            'sales_price': 'What is the sales price?',
        }

    def check_new_item(self, question: str, file_name: str) -> str:
        """Check the input does not already match an item in the list.

        Args:
            question (str): the input question.
            file_name (str): the file to open and check.

        Returns:
            str: the user input to add to the file.

        """
        # Use loop to repeat question until valid entry.
        while True:
            user_input = input(f'{question} ')
            # Open file to check products
            with Path(f'{file_name}.txt').open() as file:
                # Strip whitespace from file and input, check line by line.
                if user_input.strip() in [line.strip() for line in file]:
                    print(
                        Fore.RED
                        + 'Item already entered. Please enter a new item.'
                        + Style.RESET_ALL
                        + '\n',
                    )
                else:
                    # If input isn't in file, break the loop.
                    break
        # Return valid input.
        return user_input

    def write_files(self) -> None:
        """Loops through each file and takes input for their entry."""
        # Loop through dictionary of file names
        for file_name, question in self.FILE_NAMES.items():
            # Open file and write to it
            with Path(f'{file_name}.txt').open('a') as file:
                # Check if it is a product (str value) or a number file (int)
                if file_name == 'product':
                    # Check if product is already in file
                    user_input = self.check_new_item(question, file_name)
                else:
                    # Validates a positive integer is entered
                    user_input = validate_input_num(question, is_int=True)
                file.write(f'{user_input}\n')

    def create_dict(self) -> dict:
        """Create a dictionary of the products to use for calculations later.

        Returns:
            dict: A dictionary of data for each product from the files.

        """
        # Initialise dict for use in calculations
        product_dict = {}

        # Open each file listed in order
        for file_name in self.FILE_NAMES:
            with Path(f'{file_name}.txt').open() as file:
                # Create a dictionary with the product name for each product
                if file_name == 'product':
                    for line in file:
                        line_name = line.strip()
                        product_dict[line_name] = {}
                else:
                    # Create key-value pairs inside that product dictionary
                    for line, product_name in zip(
                        file,
                        product_dict,
                        strict=False,
                    ):
                        line_num = int(line.strip())
                        product_dict[product_name][file_name] = line_num
        # Return the dictionary for calculations
        return product_dict

    def calculate_product_stats(self, value: dict) -> tuple:
        """Calculate revenue, cost and profit for an item.

        Args:
            value (dict): the dictionary of the product to caclulate.

        Returns:
            tuple: Revenue, cost and profit.

        """
        # Calculate the relevant values
        revenue = value['quantity'] * value['sales_price']
        cost = value['quantity'] * value['purchase_price']

        # Remove cashback from revenue
        if revenue > self.UPPER_CASHBACK:
            revenue -= 3000
        if self.LOWER_CASHBACK < revenue <= self.UPPER_CASHBACK:
            revenue -= 150

        # Calculate profit
        profit = revenue - cost

        return revenue, cost, profit

    def compile_stats(self, product_dict: dict) -> int:
        """Compile stats about the products and display them.

        Args:
            product_dict (dict): dictionary of products for calculations.

        Returns:
            int: total profit of all calculations.

        """
        # Print initial table layout
        print(
            f'{"Product":<15}'
            + Fore.YELLOW
            + f'| {"Revenue":<15} '
            + Fore.RED
            + f'| {"Cost":<15} '
            + Fore.GREEN
            + f'| {"Profit"}'
            + Style.RESET_ALL,
        )
        print('-------------------------------------------------------------')

        # Initialise values for totals
        total_revenue = 0
        total_cost = 0
        total_profit = 0

        # Loop through the keys and values for each product
        for key, value in product_dict.items():
            revenue, cost, profit = self.calculate_product_stats(value)

            # Add to totals
            total_revenue += revenue
            total_cost += cost
            total_profit += profit

            # Print data for each product
            print(f'{key:<15}| ${revenue:<15}| ${cost:<15}| ${profit}')

        # Print totals
        print('-------------------------------------------------------------')
        print(
            f'{"Total":<15}'
            + Fore.YELLOW
            + f'| ${total_revenue:<15}'
            + Fore.RED
            + f'| ${total_cost:<15}'
            + Fore.GREEN
            + f'| ${total_profit}\n'
            + Style.RESET_ALL,
        )

        # Return profit for final message
        return total_profit

    def run(self) -> None:
        """Create files from input data and display financial data."""
        print(
            'This app will calculate sales statistics from product info.\n'
            'Please input positive whole numbers for'
            ' numeric values, ex. "12".\n',
        )

        # Check number of products is a positive integer
        item_num = validate_input_num(
            'How many items are you entering?',
            is_int=True,
        )

        # Loop and add data for each item to be entered
        for item in range(item_num):
            print(f'Please enter details for product {item + 1}.\n')
            self.write_files()

        # Create a dictionary of data from the files
        product_dict = self.create_dict()

        # Calculate the profit of each item and the totals
        final_profit = self.compile_stats(product_dict)

        # Display final messages
        if final_profit > 0:
            print(
                Fore.GREEN
                + "The business is on it's way to success!"
                + Style.RESET_ALL,
            )
        if final_profit == 0:
            print(
                Fore.YELLOW
                + 'Just breaking even. Hang in there!'
                + Style.RESET_ALL,
            )
        if final_profit < 0:
            print(Fore.RED + 'Time to revise your finances.' + Style.RESET_ALL)

In [5]:
sales_calculator = SalesCalculator()
sales_calculator.run()

This app will calculate sales statistics from product info.
Please input positive whole numbers for numeric values, ex. "12".

Please enter details for product 1.

Please enter details for product 2.

Item already entered. Please enter a new item.

Please input positive whole numbers, ex. "12".

Please enter details for product 3.

Please enter details for product 4.

Please enter details for product 5.

Please enter details for product 6.

Product        | Revenue         | Cost            | Profit
-------------------------------------------------------------
Mattress       | $12180          | $7560           | $4620
Bed            | $3324           | $2598           | $726
Pillow         | $2939406        | $2796282        | $143124
Door           | $140770         | $128920         | $11850
Table          | $9113408        | $459208         | $8654200
Chairs         | $57273          | $45621          | $11652
-------------------------------------------------------------
Total      